In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.6 MB/s eta 0:00:00


In [2]:
import pandas as pd

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

import transformers
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [5]:
%cd /content/drive/MyDrive/Github/sentiment-analysis

train_set = pd.read_csv('data/병합데이터셋.csv', index_col=0)
train_set.sample(n=5)

/content/drive/MyDrive/Github/sentiment-analysis


,sentence,emotion
20995,누구 맘대로 내 성장판의 희망을 꺽고 있어?,분노
36251,아빠~,중립
26729,친구가 파혼을 당해서 당황스러워.,놀람
7276,무슨 백화점 홍삼은 금을 바른건가가요?!,놀람
21811,짝이 된 친구와 친해지려고 노력했지만 친구는 날 무시해서 슬퍼.,슬픔


In [6]:
train_set['emotion'].unique()

array(['분노', '행복', '불안', '놀람', '슬픔', '중립'], dtype=object)

In [10]:
# 감정을 정수 라벨로 변경
emotions = {'행복': 0, '불안': 1, '놀람': 2, '슬픔': 3, '분노': 4, '중립': 5}
train_set['emotion'] = train_set.emotion.map(emotions)

train_set.sample(n=5)

,sentence,emotion
15233,뻔뻔함의 극치 우병우,4
27294,그러게 말이에요. 선생님은 이런 거 딱 질색이시죠?,5
34948,친구가 노후 자금 마련을 도와줘서 노후 걱정이 없어. 마음이 아주 든든해.,0
13473,뭐가요.,5
11452,나는 아이를 세 명 이상 낳을 거야.,0


In [8]:
# torch.utils.data.Dataset 상속
class mscDataset(Dataset):  #multi-label sentiment classification
  def __init__(self, csv_file):
    # 결측치 제거
    self.dataset = csv_file
    # 중복값 제거
    self.dataset.drop_duplicates(subset=['sentence'], inplace=True)
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

    print(self.dataset.describe())

#   def __init__(self, csv_file):
#     self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")
#     print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:2].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [11]:
train_data, test_data = train_test_split(train_set, test_size=0.2, random_state=0)
train_dataset = mscDataset(train_data)
test_dataset = mscDataset(test_data)

             emotion
count  113866.000000
mean        3.110077
std         1.702035
min         0.000000
25%         2.000000
50%         3.000000
75%         5.000000
max         5.000000
            emotion
count  28467.000000
mean       3.126427
std        1.700612
min        0.000000
25%        2.000000
50%        3.000000
75%        5.000000
max        5.000000


In [12]:
device = torch.device("cuda") #(colab)그냥 
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=6).to(device)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [13]:
# 하이퍼 파라미터
epochs = 3
batch_size = 16

In [14]:
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
# 학습
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

  0%|          | 0/7117 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Batch Loss: 174.60304415225983 Accuracy: tensor(0.2837, device='cuda:0')
Batch Loss: 345.9776386022568 Accuracy: tensor(0.2900, device='cuda:0')
Batch Loss: 511.7890565395355 Accuracy: tensor(0.3152, device='cuda:0')
Batch Loss: 673.681477189064 Accuracy: tensor(0.3320, device='cuda:0')
Batch Loss: 832.1879050731659 Accuracy: tensor(0.3469, device='cuda:0')
Batch Loss: 985.9594702720642 Accuracy: tensor(0.3586, device='cuda:0')
Batch Loss: 1142.069910287857 Accuracy: tensor(0.3665, device='cuda:0')
Batch Loss: 1290.704150378704 Accuracy: tensor(0.3760, device='cuda:0')
Batch Loss: 1443.3577674031258 Accuracy: tensor(0.3814, device='cuda:0')
Batch Loss: 1594.7030233740807 Accuracy: tensor(0.3851, device='cuda:0')
Batch Loss: 1740.3050698637962 Accuracy: tensor(0.3904, device='cuda:0')
Batch Loss: 1888.1700080037117 Accuracy: tensor(0.3942, device='cuda:0')
Batch Loss: 2038.436638891697 Accuracy: tensor(0.3956, device='cuda:0')
Batch Loss: 2185.3498917222023 Accuracy: tensor(0.3984, devi

  0%|          | 0/7117 [00:00<?, ?it/s]

Batch Loss: 127.63008695840836 Accuracy: tensor(0.5125, device='cuda:0')
Batch Loss: 256.15537559986115 Accuracy: tensor(0.5119, device='cuda:0')
Batch Loss: 384.1029177904129 Accuracy: tensor(0.5083, device='cuda:0')
Batch Loss: 512.0457469820976 Accuracy: tensor(0.5091, device='cuda:0')
Batch Loss: 638.9629372358322 Accuracy: tensor(0.5105, device='cuda:0')
Batch Loss: 766.3194610476494 Accuracy: tensor(0.5093, device='cuda:0')
Batch Loss: 890.6922461986542 Accuracy: tensor(0.5130, device='cuda:0')
Batch Loss: 1016.1859129667282 Accuracy: tensor(0.5128, device='cuda:0')
Batch Loss: 1148.0887361168861 Accuracy: tensor(0.5106, device='cuda:0')
Batch Loss: 1273.706263422966 Accuracy: tensor(0.5106, device='cuda:0')
Batch Loss: 1402.2711577415466 Accuracy: tensor(0.5106, device='cuda:0')
Batch Loss: 1529.0774171948433 Accuracy: tensor(0.5113, device='cuda:0')
Batch Loss: 1654.1336635947227 Accuracy: tensor(0.5126, device='cuda:0')
Batch Loss: 1779.6164351701736 Accuracy: tensor(0.5130, d

  0%|          | 0/7117 [00:00<?, ?it/s]

Batch Loss: 117.03224951028824 Accuracy: tensor(0.5612, device='cuda:0')
Batch Loss: 225.81474936008453 Accuracy: tensor(0.5844, device='cuda:0')
Batch Loss: 335.2595000267029 Accuracy: tensor(0.5923, device='cuda:0')
Batch Loss: 445.3264354467392 Accuracy: tensor(0.5944, device='cuda:0')
Batch Loss: 556.7654874324799 Accuracy: tensor(0.5908, device='cuda:0')
Batch Loss: 665.3947442770004 Accuracy: tensor(0.5906, device='cuda:0')
Batch Loss: 777.4592826962471 Accuracy: tensor(0.5889, device='cuda:0')
Batch Loss: 884.7314205765724 Accuracy: tensor(0.5918, device='cuda:0')
Batch Loss: 997.3263311684132 Accuracy: tensor(0.5910, device='cuda:0')
Batch Loss: 1113.6396072208881 Accuracy: tensor(0.5880, device='cuda:0')
Batch Loss: 1222.009627431631 Accuracy: tensor(0.5899, device='cuda:0')
Batch Loss: 1330.8938225209713 Accuracy: tensor(0.5900, device='cuda:0')
Batch Loss: 1440.7423594892025 Accuracy: tensor(0.5908, device='cuda:0')
Batch Loss: 1553.8439778983593 Accuracy: tensor(0.5897, dev

In [16]:
losses, accuracies

([10280.558019697666, 8635.074956178665, 7780.665116965771],
 [tensor(0.4428, device='cuda:0'),
  tensor(0.5431, device='cuda:0'),
  tensor(0.5957, device='cuda:0')])

In [17]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/1780 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Accuracy: tensor(0.6080, device='cuda:0')


In [ ]:
torch.save(model, 'model/koelectra-v1.pt')

In [ ]:
import os

size2 = os.path.getsize('model/koelectra-v1.pt') / (1024*1024) # mb 단위
print(f"Model size: {size2:.2f} MB")